In [1]:
import numpy as np

from pint import UnitRegistry
u = UnitRegistry()
u.load_definitions('./mis-defs.txt') 

import matplotlib.pyplot as plt

%matplotlib inline

# Funciones (Python)

Para definir una función se usa la palabra clave *def* seguido del nombre asignado a la misma, luego los argumentos entre paréntesis y finalmente dos puntos. En las lineas siguientes van las instrucciones que componen la función. El bloque dentro de la función queda definido sólo por la indentación (no hay llaves ni otro tipo de elemento para señalar el conjunto de instrucciones que componen la función). Finalmente los valores que devuelve la función se colocan tras la palabra clave ***return*** en la última linea. 

La estructura es la siguiente (más adelante hay ejemplos ejecutables):

```  python
def nombre_de_la_funcion(argumento1, argumento2):
   instrucción
   instrucción
   instrucción
   return resultados
``` 


Vamos a definir una función que permita calcular la evolución de los decaimientos de una muestra radiactiva

$N = N_0 \cdot e^{-\lambda t}$

En vez de usar $\lambda$ vamos a utilizar $\tau$

$$N = N_0 \cdot e^{-\frac{ln(2) \cdot t}{\tau}}$$


In [2]:
def n(n0, tau, t):
    exponente = - np.log(2) * t /tau
    return n0 * np.exp(exponente)

La función exponencial del ejemplo proviene de la librería ***Numpy*** que importamos originalmente como ***np***.
Probemos con algunos valores elegidos al azar:

In [12]:
n0 = 100
tau = 24 * u('hours')

Verificamos que ingresamos los valores adecuados (y que reconoce las unidades)

In [15]:
print("N0: ", n0)
print("Tau: ", tau.to('days'))

N0:  100
Tau:  1.0 day


Podemos calcular el número de nucleos en distintos tiempos (variando t1):

In [17]:
t1 = 24 * 60 * u('min')
n_a = n(n0, tau, t1)
n_a

50.0 <Unit('dimensionless')>

**Fíjense que t1 puede tener cualquier unidad de tiempo (microsegundos, días, etc.), no es necesario que coincida con las unidades de Tau.** 

## Uso de vectores y unidades para graficar

Generamos el vector de tiempos y le otorgamos unidades.

In [ ]:
vector_t = np.arange(0.0, 10.0, 0.01) * u('min')

Verificamos algún elemento del vector

In [ ]:
vector_t[20]

Generamos el vector de nucleos. Definimos nuevamente n0 y tau para tener las variables más cerca y poder cambiarlas más facilmente.

In [ ]:
n0 = 1000
tau = 2 * u('min')

vector_n = n(n0, tau, vector_t)

Mostramos algún elemento para verificar...

In [ ]:
vector_n[10]

## Graficación

Graficamos del siguiente modo. **Da un aviso de que la gráfica se realiza habiendo quitado las unidades (no importa, nos sirve igual)**

In [ ]:
fig, ax = plt.subplots()
ax.plot(vector_t, vector_n)

ax.set(xlabel='Tiempo', ylabel='Nucleos', title='Decaimiento')
ax.grid()

plt.show()

## Algunas funciones útiles

In [ ]:
def N(n0, tmedio, t):
    '''Calcula la exponencial decreciente que describen los núcleos radiactivos al decaer'''
    lmbd = np.log(2)/tmedio
    exponente = - lmbd * t
    return n0 * np.exp(exponente)

def A(a0, tmedio, t):
    '''Calcula la exponencial decreciente que describe la actividad'''
    lmbd = np.log(2)/tmedio
    exponente = - lmbd * t
    return a0 * np.exp(exponente)

def AtoN(a, tmedio):
    '''Convierte la actividad instantánea en el número de núcleos presentes en la muestra en ese instante'''
    lmbd = np.log(2)/tmedio
    return a/lmbd
    
def concAct(a, vol):
    '''Calcula la concentración de actividad en una solución
    a: actividad
    vol: volumen de solución'''
    return a/vol

def mostrar(numero,precision=2):
    '''Esta función permite darle formato científico a un número. El argumento
    *precision* es el número de decimales con que se verá.
    '''    
    print( "{:.{}e}".format(numero, precision ) )

### Ayuda

La documentación presente en el inicio de cada función se denomina docstring. La mayoría de los paquetes de Python tienen todas sus funciones con docstrings así que se puede invocar la ayuda de todas ellas en cualquier momento. Y se puede invocar de dos maneras:
1. Con *Shift+Tab* luego del nombre de la función. Se abre una ventanita que a su vez puede ser expandida pulsando sobre el símbolo + (que aparece en esa ventanita, arriba a la derecha).
2. Con la orden *help(nombreDeLaFunción)*

In [ ]:
help(concAct)

### Uso de constantes del paquete Scipy.

De acuerdo al modo en que se importen las constantes, varía la sintaxis para invocarlas. En la celda siguiente se muestra cómo importar todas las constantes del paquete *scipy*, y asignar una sola de ellas a una variable.

In [ ]:
from scipy import constants

Avogadro = constants.Avogadro
Avogadro

Podemos importar sólo la constante de nuestro interés.

In [ ]:
from scipy.constants import Avogadro

Avogadro

## Cálculo de masa de una muestra

In [ ]:
from scipy.constants import Avogadro

def masa(n, masaMolar):
    '''Devuelve la masa correspondiente al número de nucleos presentes en la muestra (n) y la masa molar del RN'''
    return n * masaMolar / Avogadro

### Ejemplo

Un ejemplo para verificar que funciona. Masa de Tc99m presentes en una muestra cuya actividad es de 180 mCi:

In [ ]:
# datos

A = 180 * u('mCi')
tmedio = 6.02 * u('hour')
masaTc99m = 99 * u('g')

Primero calculamos el número de nucleos y luego finalmente la masa correspondiente.

In [ ]:
N = AtoN(A,tmedio)
m_muestra = masa(N, masaTc99m)

## Presentación de resultados

Las variables se pueden mostrar con distintas notaciones. A continuación algunos ejemplos:

* Sin procesar

In [ ]:
m_muestra

* en gramos y submúltiplos:

In [ ]:
print(m_muestra.to('g'))
print(m_muestra.to('pg'))
print(m_muestra.to('ng'))

* usando la función *mostrar* para ver en notación científica con diverso número de cifras significativas:

In [ ]:
mostrar(m_muestra.to('g'))
mostrar(m_muestra.to('pg'),5)
mostrar(m_muestra.to('ng'),3)